In [ ]:
#from elasticsearch_dsl import Search, Q, Range, connections
#from elasticsearch.client import Elasticsearch
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
from openai import OpenAI
import numpy as np
import ast

In [ ]:
EVENTS_FOLDER = "../data/events/"

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option('display.max_colwidth', None)

In [ ]:
df_events = pd.read_pickle(f"{EVENTS_FOLDER}df_events 2025-01-14_18-50-06.pkl")

In [ ]:
df_events

In [ ]:
df_events.iloc[24].StartDateTime

In [ ]:
len(df_events[(df_events.StartDate.isna())])

In [ ]:
df_events.lat.isna().value_counts()

In [ ]:
df_events.columns

In [ ]:
df_events.drop(columns=['lat_decimal', 'lat_str', 'lat_decimal_precision'], inplace=True, errors='ignore')

In [ ]:
df_events.info()

In [ ]:
df_football = pd.read_csv(f"{EVENTS_FOLDER}Football matches.csv", dtype={'lat': float, 'lng': float})

In [ ]:
df_football.StartDate = pd.to_datetime(df_football.StartDate).dt.date.astype(str)

In [ ]:
df_football.info()

In [ ]:
def change_football_hm_to_standard_format(s):
    spl = s.split(":")
    hour= int(spl[0])
    if spl[1]=="15":
        minute=""
    elif spl[1]=="30":
        minute=",5"
    elif spl[1]=="45":
        minute=""
        hour+=1
    else: 
        minute=""
    return "".join([str(hour), minute])
        

In [ ]:
df_football

In [ ]:
df_football.StartTime = df_football.StartTime.apply(change_football_hm_to_standard_format)

In [ ]:
df_football

In [ ]:
df_football['EventCategory'] = 'Football match'

In [ ]:
df_football['Source']= 'Football matches'

In [ ]:
# assume game duration of 2 hours
df_football['EndTime'] = (df_football.StartTime.str.replace(",", ".").astype(float)+2).astype(str)

In [ ]:
df_football['EndTime']= df_football['EndTime'].str.replace(".", ",").str.replace(",0", "")

In [ ]:
df_football

In [ ]:
df_events = pd.concat([df_events, df_football], ignore_index=True)

In [ ]:
df_events.Source.value_counts()

In [ ]:
flt  = df_events.StartTime.str.contains(":", na=False)
values = df_events.loc[flt, "StartTime"].apply(change_football_hm_to_standard_format)
df_events.loc[flt, "StartTime"] = values


In [ ]:
#pd.concat([df_events.loc[flt, "StartTime"], df_events.loc[flt, "StartTime"].apply(change_football_hm_to_standard_format)], axis=1).drop_duplicates()

In [ ]:
df_events.StartTime.value_counts()

In [ ]:
flt  = df_events.EndTime.str.contains(":", na=False)
values = df_events.loc[flt, "EndTime"].apply(change_football_hm_to_standard_format)
df_events.loc[flt, "EndTime"] = values


# get StartDateTime and EndDateTime for all events

## what is the most common start time of gema events

In [ ]:
df_meslis = pd.read_pickle(f"{EVENTS_FOLDER}meslis/df_meslis 2024-12-19_12-41-21.pkl")
df_meslis.customers.drop_duplicates()
GemaDates = df_meslis.loc[df_meslis.customers.str.contains("{'name': 'GEMA', 'matchedEventIds':"), ["startDate", "endDate"]]

In [ ]:
GemaDates.startDate = GemaDates.startDate.apply(lambda x: x[11:]).str.replace("Z", "").str.replace("+02:00", "").str.replace("+01:00", "")
flt  = ~GemaDates.endDate.isna()
GemaDates.endDate[flt] = GemaDates.endDate[flt].apply(lambda x: x[11:]).str.replace("Z", "").str.replace("+02:00", "").str.replace("+01:00", "")

In [ ]:
GemaDates.endDate = GemaDates.endDate.fillna("X")

In [ ]:
GemaDates.value_counts()

In [ ]:
GemaEndTimes = GemaEndDates.apply(lambda x: x[11:]).str.replace("Z", "").str.replace("+02:00", "").str.replace("+01:00", "")

In [ ]:
GemaEndTimes.value_counts()

In [ ]:
#20 00 is the most common start time, then comes 19, andd then 10
# finish is usually at midnight

In [ ]:
# GemaStartTimes.value_counts()

In [ ]:
df_events.loc[(df_events.StartTime=="10:00") & (df_events.Source=="browse_ai")]

In [ ]:
df_events.loc[df_events.Source=="Gema", "StartTime"] = "20"
df_events.loc[df_events.Source=="Gema", "EndTime"] = "0"

In [ ]:
df_events.loc[(df_events.Source=="facebook.com") & ~df_events.StartTime.isna()]

In [ ]:
df_events.loc[df_events.Source=="facebook.com" ,['StartTime']].value_counts()

In [ ]:
flt = df_events.StartTime.isna()
df_events.loc[flt, "Source"].value_counts()

In [ ]:
df_events.groupby(["Source", "StartTime"]).size()

## apply transformations

In [ ]:
for col in ['StartDate', "EndDate"]: 
    df_events[col]= df_events[col].astype(str)
    flt = df_events[col].str.contains("00:00:00", na=False)
    df_events.loc[flt, col] = df_events.loc[flt, col].str.replace("00:00:00", "").str.strip()

    flt = ~df_events[col].str.contains("20", na=False)
    df_events.loc[flt, col] = pd.NA




In [ ]:
for col in ["StartTime", "EndTime"]:
    df_events[col].astype(str)
    df_events[col]= df_events[col].str.strip()
    flt = ~df_events[col].str.contains("\d", regex=True,na=False)
    df_events.loc[flt, col]=pd.NA

    flt = df_events[col].str.contains(",5", na=False)
    df_events.loc[flt, col] = df_events.loc[flt, col].str.replace(",5", ":30")
    flt2 = (~flt) & (~df_events[col].isna())
    df_events.loc[flt2, col] = df_events.loc[flt2, col]+":00"

In [ ]:
df_events['StartDate'].str.len().value_counts()

In [ ]:
df_events['EndDate'].str.len().value_counts()

In [ ]:
df_events.StartTime

In [ ]:
for DateTime, Date, Time in [("StartDateTime", "StartDate", "StartTime"), ("EndDateTime", "EndDate", "EndTime")]:
    flt = df_events[DateTime].isna() & ~df_events[Date].isna() & ~df_events[Time].isna()
    df_events.loc[flt, DateTime] =  pd.to_datetime(df_events.loc[flt, Date] + " " + df_events.loc[flt, Time])

In [ ]:
df_events.StartDateTime.isna().value_counts()

In [ ]:
df_events.iloc[24].StartDateTime

In [ ]:
df_events.iloc[24].EndDateTime

In [ ]:
df_events[df_events.Source=="facebook.com"]

In [ ]:
df_events.EndDateTime.isna().value_counts()

In [ ]:
df_events.to_pickle(f"{EVENTS_FOLDER}/df_events {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")